In [4]:
import strawberryfields as sf
from strawberryfields.ops import *
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Parameters
cutoff = 32
s = 1.0
a_target = 2.0
p = 2
x_vals = np.linspace(-10, 10, 200)

# Approximate cost unitary: use displacement + Kgate + CubicPhase
def apply_cost_gate(q, eta):
    Dgate(a_target) | q
    CubicPhase(eta) | q
    Kgate(eta)      | q  # adds a quartic-like effect (approximates x^4)
    Dgate(-a_target) | q

def apply_mixer_gate(q, gamma):
    Fourier() | q
    Kgate(gamma) | q
    Fourier().H | q

# CV-QAOA circuit
def cv_qaoa_sf(params):
    gamma_list = params[:p]
    eta_list = params[p:]

    eng = sf.Engine("fock", backend_options={"cutoff_dim": cutoff})
    prog = sf.Program(1)

    with prog.context as q:
        Sgate(s) | q[0]

        for i in range(p):
            apply_cost_gate(q[0], eta_list[i])
            apply_mixer_gate(q[0], gamma_list[i])

    state = eng.run(prog).state

    # Compute ⟨x⟩ from Wigner
    wigner = state.wigner(0, x_vals, np.zeros_like(x_vals))
    dx = x_vals[1] - x_vals[0]
    x_expect = np.sum(x_vals * wigner[:, 0]) * dx

    return (x_expect - a_target)**6

# Visualize final Wigner function
def plot_final_wigner(params):
    gamma_list = params[:p]
    eta_list = params[p:]

    eng = sf.Engine("fock", backend_options={"cutoff_dim": cutoff})
    prog = sf.Program(1)

    with prog.context as q:
        Sgate(s) | q[0]
        for i in range(p):
            apply_cost_gate(q[0], eta_list[i])
            apply_mixer_gate(q[0], gamma_list[i])

    state = eng.run(prog).state
    wigner = state.wigner(0, x_vals, np.zeros_like(x_vals))

    plt.contourf(x_vals, x_vals, wigner, levels=100, cmap='RdBu_r')
    plt.colorbar()
    plt.title("Final Wigner Function")
    plt.xlabel("x")
    plt.ylabel("p")
    plt.tight_layout()
    plt.show()


In [5]:
init_params = [0.3] * (2 * p)  # gamma + eta

res = minimize(cv_qaoa_sf, init_params, method="COBYLA", options={"maxiter": 100})
print("Optimal parameters:", res.x)
plot_final_wigner(res.x)


NameError: name 'CubicPhase' is not defined